<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/master/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import numpy as np
import os
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

## Reproduce baseline of Naive Bayes approach

### Data

Loading the Dataset

In [ ]:
SRC_DIR = r'D:\projects\ant\Labelled Datatset-002\\'
FNAME = SRC_DIR + 'Combined_FAANG_percentage_previous.csv'
DO_DROP_NEUTRAL_LABELS = True

df = pd.read_csv(FNAME)
df = df.sample(frac=1, random_state=42)
df.head()

,symbol,message,datetime,user,message_id,Date,Time,label
1804152,GOOGL,shit 1 5 play personally thanking ceo goog wmt...,2020-03-14 04:08:25,1357898,200350334,2020-03-14,04:08:25,-1
492417,AAPL,aapl apple space program,2018-05-18 08:03:45,1247534,123934647,2018-05-18,08:03:45,0
520223,AAPL,watch downside price targets aapl,2018-03-23 12:14:25,55919,117351397,2018-03-23,12:14:25,-1
1623572,AMZN,quot lcc 007 amzn layin wood quot,2014-06-06 19:18:02,162320,23588295,2014-06-06,19:18:02,1
392407,AAPL,aapl stock trading vehicle growth trade back f...,2018-11-07 22:45:48,1395651,144365846,2018-11-07,22:45:48,1


Limiting dataset to 1 year

In [ ]:
df.drop(df[df['Date'] <= '2019-07-20'].index, inplace = True)

Dropping Neutral labels

In [ ]:
df.drop(df[df['label']==0].index, inplace= True)
df['label'].replace({-1:0}, inplace = True)

In [ ]:
df["label"].value_counts()

1    238950
0    170484
Name: label, dtype: int64

### Splitting Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], random_state = 42, test_size = 0.1)

### TF-IDF Vectorizer

In [ ]:
tf_idf = TfidfVectorizer(use_idf=True, ngram_range=(2, 2))

In [ ]:
#Extracting features of the training data
train_features = tf_idf.fit_transform(X_train)

#Extracting features of the test data
test_features = tf_idf.transform(X_test)

### Training the Model

In [ ]:
model = MultinomialNB(alpha=0.1) 
model.fit(train_features, y_train)
model.score(test_features, y_test)

0.6199687377881985

### Prediction on Test set

In [ ]:
y_pred = model.predict(test_features)

### Classification Report

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.40      0.47     17190
           1       0.64      0.78      0.70     23754

    accuracy                           0.62     40944
   macro avg       0.60      0.59      0.59     40944
weighted avg       0.61      0.62      0.61     40944



## Improve naive bayes approach

In [ ]:
# - restore stop words
# - appended symbol at the end of text
# - tuned parameter alpha for naive bayes

### Restore stop words

In [ ]:
DATA_PATH = r"D:\projects\ant\stocktwits_finsentiment_analysis-20221130T194929Z-001\Datasets\2022-11-29__sfa-ds-010__faang_raw\\"

dfs = []
for f in [os.path.join(DATA_PATH, "AAPL_RmSW=0_RmRep=0_1y.csv.gz"), 
          os.path.join(DATA_PATH, "AMZN_RmSW=0_RmRep=0_1y.csv.gz"), 
          os.path.join(DATA_PATH, "FB_RmSW=0_RmRep=0_1y.csv.gz"),
          os.path.join(DATA_PATH, "GOOGL_RmSW=0_RmRep=0_1y.csv.gz"),
          os.path.join(DATA_PATH, "NFLX_RmSW=0_RmRep=0_1y.csv.gz")]:
    df_ = pd.read_csv(f)
    dfs.append(df_)

In [ ]:
df_united = pd.concat(dfs, ignore_index=True)

In [ ]:
df_united.message_id.value_counts()

127546947    6
121014115    6
126786940    6
120180598    6
157612440    6
            ..
41024099     1
41024154     1
41024182     1
41024250     1
411119       1
Name: message_id, Length: 2406354, dtype: int64

In [ ]:
df_tmp = df_united[['message', 'message_id']].copy()

In [ ]:
df_tmp.drop_duplicates(subset=['message_id'], inplace=True)

In [ ]:
# Drop incorrect texts from initial dataset
print(df.shape)
df.dropna(subset=['message'], inplace=True)
print(df.shape)

(409434, 8)
(409434, 8)


In [ ]:
df_joined = df.merge(df_tmp, on='message_id', how='left', validate="m:1")

In [ ]:
assert len(df_joined) == len(df)

In [ ]:
assert df.message.isna().sum() == df_joined.message_y.isna().sum()

### Add ticker name to the end of message

In [ ]:
sym = df_joined['symbol'].str.lower().replace("googl", "goog")
df_joined['message_w_symbol'] = df_joined['message_y'] + ' ' + sym
df_joined

### Splitting Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_joined['message_w_symbol'], df_joined['label'], 
                                                                            random_state = 45, test_size = 0.1)

In [ ]:
X_train.head()

211020    nflx if pre announcement were to happen will l...
134444    spy aapl tsla msft net debt to ebitda ratio fo...
379393    the press overnight sniffed out nflx amp the s...
295742    spy happy father s day to all the wonderful fa...
316842    aapl  1 hour view from last friday midday upda...
Name: message_w_symbol, dtype: object

### TF-IDF Vectorizer

In [ ]:
tf_idf = TfidfVectorizer(use_idf=True, ngram_range=(1, 5))

In [ ]:
#Extracting features of the training data
train_features = tf_idf.fit_transform(X_train)

#Extracting features of the test data
test_features = tf_idf.transform(X_test)


### Training the Model

check dependency from hyparameter alpha

In [ ]:
for alpha in [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]:
    model = MultinomialNB(alpha=alpha)
    model.fit(train_features, y_train)

    print(alpha, model.score(test_features, y_test))

0.02 0.6317653380226651
0.04 0.6368454474404064
0.06 0.6397518561938257
0.08 0.6397762797967956
0.1 0.640240328253224
0.12 0.639947245017585
0.14 0.6395564673700664
0.16 0.6390435717076983
0.18 0.6384085580304807
0.2 0.6368454474404064


Best alpha is 0.1

### Train model on characters

In [ ]:
tf_idf2 = TfidfVectorizer(use_idf=True, ngram_range=(9, 9), analyzer='char')

In [ ]:
#Extracting features of the training data
train_features2 = tf_idf2.fit_transform(X_train)

#Extracting features of the test data
test_features2 = tf_idf2.transform(X_test)

check dependency from hyparameter alpha

In [ ]:
for alpha in [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]:
    model2 = MultinomialNB(alpha=alpha)
    model2.fit(train_features2, y_train)

    print(alpha, model2.score(test_features2, y_test))

0.02 0.6288345056662759
0.04 0.6309349355216881
0.06 0.6336215318483783
0.08 0.6334017194216491
0.1 0.635331184056272
0.12 0.6357952325127003
0.14 0.6368942946463463
0.16 0.6371629542790153
0.18 0.6380910511918718
0.2 0.636161586557249


Best alpha is 0.18

### Models blending

Enumerate parameters: alpha1, alpha2, contribution weights of two models, threshold for predict_proba()

In [ ]:
best_res = None
for alpha1 in [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]:
    model1 = MultinomialNB(alpha=alpha1)  # TODO: check other alpha
    model1.fit(train_features, y_train)
    for alpha2 in [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]:
        model2 = MultinomialNB(alpha=alpha2)  # TODO: check other alpha
        model2.fit(train_features2, y_train)
        
        for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
            q = 1 - p
            probas1 = model1.predict_proba(test_features)[:, 1]  # take class 1
            probas2 = model2.predict_proba(test_features2)[:, 1]  # take class 1
            probas_sum = p * probas1 + q * probas2

            # check dependency from threshold
            for th in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                preds = (probas_sum >= th) * 1  # convert bool array to [0, 1]
                cur_res = accuracy_score(preds, y_test)
                if best_res is None or best_res < cur_res:
                    best_res = cur_res
                print(alpha1, alpha2, p, th, cur_res, best_res)

0.02 0.02 0.0 0.0 0.585336068776866 0.585336068776866
0.02 0.02 0.0 0.1 0.6298602969910121 0.6298602969910121
0.02 0.02 0.0 0.2 0.633694802657288 0.633694802657288
0.02 0.02 0.0 0.3 0.6339146150840171 0.6339146150840171
0.02 0.02 0.0 0.4 0.6327422821414616 0.6339146150840171
0.02 0.02 0.0 0.5 0.6288345056662759 0.6339146150840171
0.02 0.02 0.0 0.6 0.6187231340367331 0.6339146150840171
0.02 0.02 0.0 0.7 0.6028477921062915 0.6339146150840171
0.02 0.02 0.0 0.8 0.5803536537710043 0.6339146150840171
0.02 0.02 0.0 0.9 0.5427901524032825 0.6339146150840171
0.02 0.02 0.0 1.0 0.4148348964439234 0.6339146150840171
0.02 0.02 0.1 0.0 0.585336068776866 0.6339146150840171
0.02 0.02 0.1 0.1 0.6299579914028918 0.6339146150840171
0.02 0.02 0.1 0.2 0.635062524423603 0.635062524423603
0.02 0.02 0.1 0.3 0.6350381008206331 0.635062524423603
0.02 0.02 0.1 0.4 0.6345984759671747 0.635062524423603
0.02 0.02 0.1 0.5 0.6303731926533802 0.635062524423603
0.02 0.02 0.1 0.6 0.6194069949198906 0.635062524423603
0.0

0.02 0.04 0.3 0.0 0.585336068776866 0.6403868698710434
0.02 0.04 0.3 0.1 0.6274667838999609 0.6403868698710434
0.02 0.04 0.3 0.2 0.6360394685423993 0.6403868698710434
0.02 0.04 0.3 0.3 0.639116842516608 0.6403868698710434
0.02 0.04 0.3 0.4 0.6386283704572099 0.6403868698710434
0.02 0.04 0.3 0.5 0.6360150449394295 0.6403868698710434
0.02 0.04 0.3 0.6 0.6240719030871434 0.6403868698710434
0.02 0.04 0.3 0.7 0.6050459163735834 0.6403868698710434
0.02 0.04 0.3 0.8 0.5775205158264948 0.6403868698710434
0.02 0.04 0.3 0.9 0.5299921844470497 0.6403868698710434
0.02 0.04 0.3 1.0 0.4147616256350137 0.6403868698710434
0.02 0.04 0.4 0.0 0.585336068776866 0.6403868698710434
0.02 0.04 0.4 0.1 0.62771101992966 0.6403868698710434
0.02 0.04 0.4 0.2 0.6368210238374366 0.6403868698710434
0.02 0.04 0.4 0.3 0.6395320437670965 0.6403868698710434
0.02 0.04 0.4 0.4 0.6393610785463072 0.6403868698710434
0.02 0.04 0.4 0.5 0.6352334896443923 0.6403868698710434
0.02 0.04 0.4 0.6 0.6255861664712778 0.64038686987104

0.02 0.06 0.6 0.0 0.585336068776866 0.6403868698710434
0.02 0.06 0.6 0.1 0.6275889019148104 0.6403868698710434
0.02 0.06 0.6 0.2 0.6373583431027745 0.6403868698710434
0.02 0.06 0.6 0.3 0.6406066822977726 0.6406066822977726
0.02 0.06 0.6 0.4 0.6401670574443142 0.6406066822977726
0.02 0.06 0.6 0.5 0.6355021492770614 0.6406066822977726
0.02 0.06 0.6 0.6 0.6262456037514654 0.6406066822977726
0.02 0.06 0.6 0.7 0.6070242282141461 0.6406066822977726
0.02 0.06 0.6 0.8 0.5794011332551778 0.6406066822977726
0.02 0.06 0.6 0.9 0.5341197733489644 0.6406066822977726
0.02 0.06 0.6 1.0 0.4147616256350137 0.6406066822977726
0.02 0.06 0.7 0.0 0.585336068776866 0.6406066822977726
0.02 0.06 0.7 0.1 0.6294206721375537 0.6406066822977726
0.02 0.06 0.7 0.2 0.6377246971473232 0.6406066822977726
0.02 0.06 0.7 0.3 0.6408753419304416 0.6408753419304416
0.02 0.06 0.7 0.4 0.6395320437670965 0.6408753419304416
0.02 0.06 0.7 0.5 0.6350136772176631 0.6408753419304416
0.02 0.06 0.7 0.6 0.6261479093395858 0.64087534193

0.02 0.08 0.9 0.0 0.585336068776866 0.6410463071512309
0.02 0.08 0.9 0.1 0.6337436498632278 0.6410463071512309
0.02 0.08 0.9 0.2 0.6367477530285268 0.6410463071512309
0.02 0.08 0.9 0.3 0.6385550996483002 0.6410463071512309
0.02 0.08 0.9 0.4 0.6370896834701055 0.6410463071512309
0.02 0.08 0.9 0.5 0.6328644001563111 0.6410463071512309
0.02 0.08 0.9 0.6 0.6246092223524814 0.6410463071512309
0.02 0.08 0.9 0.7 0.6065601797577178 0.6410463071512309
0.02 0.08 0.9 0.8 0.5833577569363032 0.6410463071512309
0.02 0.08 0.9 0.9 0.5462827276279797 0.6410463071512309
0.02 0.08 0.9 1.0 0.4147616256350137 0.6410463071512309
0.02 0.08 1.0 0.0 0.585336068776866 0.6410463071512309
0.02 0.08 1.0 0.1 0.6340367330988668 0.6410463071512309
0.02 0.08 1.0 0.2 0.6370896834701055 0.6410463071512309
0.02 0.08 1.0 0.3 0.6378956623681126 0.6410463071512309
0.02 0.08 1.0 0.4 0.6369919890582258 0.6410463071512309
0.02 0.08 1.0 0.5 0.6317653380226651 0.6410463071512309
0.02 0.08 1.0 0.6 0.6231193825713169 0.64104630715

0.02 0.12 0.1 0.0 0.585336068776866 0.6410463071512309
0.02 0.12 0.1 0.1 0.6150595935912466 0.6410463071512309
0.02 0.12 0.1 0.2 0.6253175068386089 0.6410463071512309
0.02 0.12 0.1 0.3 0.6330842125830403 0.6410463071512309
0.02 0.12 0.1 0.4 0.6374316139116842 0.6410463071512309
0.02 0.12 0.1 0.5 0.6371873778819851 0.6410463071512309
0.02 0.12 0.1 0.6 0.629347401328644 0.6410463071512309
0.02 0.12 0.1 0.7 0.6110541227041814 0.6410463071512309
0.02 0.12 0.1 0.8 0.5775449394294646 0.6410463071512309
0.02 0.12 0.1 0.9 0.5165347792106292 0.6410463071512309
0.02 0.12 0.1 1.0 0.41466393122313405 0.6410463071512309
0.02 0.12 0.2 0.0 0.585336068776866 0.6410463071512309
0.02 0.12 0.2 0.1 0.616305197342712 0.6410463071512309
0.02 0.12 0.2 0.2 0.6259769441187965 0.6410463071512309
0.02 0.12 0.2 0.3 0.6354533020711215 0.6410463071512309
0.02 0.12 0.2 0.4 0.63857952325127 0.6410463071512309
0.02 0.12 0.2 0.5 0.6384574052364205 0.6410463071512309
0.02 0.12 0.2 0.6 0.6322782336850332 0.64104630715123

...

0.2 0.1 0.2 0.0 0.585336068776866 0.6430978898007034
0.2 0.1 0.2 0.1 0.615499218444705 0.6430978898007034
0.2 0.1 0.2 0.2 0.6242917155138726 0.6430978898007034
0.2 0.1 0.2 0.3 0.6325224697147324 0.6430978898007034
0.2 0.1 0.2 0.4 0.6385550996483002 0.6430978898007034
0.2 0.1 0.2 0.5 0.6381887456037515 0.6430978898007034
0.2 0.1 0.2 0.6 0.6316187964048456 0.6430978898007034
0.2 0.1 0.2 0.7 0.6142291910902696 0.6430978898007034
0.2 0.1 0.2 0.8 0.5788149667839 0.6430978898007034
0.2 0.1 0.2 0.9 0.5166080500195389 0.6430978898007034
0.2 0.1 0.2 1.0 0.41466393122313405 0.6430978898007034
0.2 0.1 0.3 0.0 0.585336068776866 0.6430978898007034
0.2 0.1 0.3 0.1 0.6152549824150059 0.6430978898007034
0.2 0.1 0.3 0.2 0.6236567018366549 0.6430978898007034
0.2 0.1 0.3 0.3 0.6318630324345448 0.6430978898007034
0.2 0.1 0.3 0.4 0.6381643220007815 0.6430978898007034
0.2 0.1 0.3 0.5 0.6395564673700664 0.6430978898007034
0.2 0.1 0.3 0.6 0.6343053927315357 0.6430978898007034
0.2 0.1 0.3 0.7 0.616085384915982

0.2 0.12 0.5 0.0 0.585336068776866 0.6430978898007034
0.2 0.12 0.5 0.1 0.6127393513091052 0.6430978898007034
0.2 0.12 0.5 0.2 0.6213608831574834 0.6430978898007034
0.2 0.12 0.5 0.3 0.6283216100039077 0.6430978898007034
0.2 0.12 0.5 0.4 0.637236225087925 0.6430978898007034
0.2 0.12 0.5 0.5 0.6407043767096522 0.6430978898007034
0.2 0.12 0.5 0.6 0.6395076201641267 0.6430978898007034
0.2 0.12 0.5 0.7 0.6217028135990621 0.6430978898007034
0.2 0.12 0.5 0.8 0.5838462289957015 0.6430978898007034
0.2 0.12 0.5 0.9 0.5145808909730364 0.6430978898007034
0.2 0.12 0.5 1.0 0.41466393122313405 0.6430978898007034
0.2 0.12 0.6 0.0 0.585336068776866 0.6430978898007034
0.2 0.12 0.6 0.1 0.6129103165298945 0.6430978898007034
0.2 0.12 0.6 0.2 0.6219226260257913 0.6430978898007034
0.2 0.12 0.6 0.3 0.6277842907385698 0.6430978898007034
0.2 0.12 0.6 0.4 0.6361127393513091 0.6430978898007034
0.2 0.12 0.6 0.5 0.6410218835482611 0.6430978898007034
0.2 0.12 0.6 0.6 0.6406066822977726 0.6430978898007034
0.2 0.12 0.6

0.2 0.14 0.8 0.0 0.585336068776866 0.6430978898007034
0.2 0.14 0.8 0.1 0.6128858929269245 0.6430978898007034
0.2 0.14 0.8 0.2 0.6219714732317312 0.6430978898007034
0.2 0.14 0.8 0.3 0.6271981242672919 0.6430978898007034
0.2 0.14 0.8 0.4 0.6335482610394686 0.6430978898007034
0.2 0.14 0.8 0.5 0.6395320437670965 0.6430978898007034
0.2 0.14 0.8 0.6 0.6425605705353654 0.6430978898007034
0.2 0.14 0.8 0.7 0.6274667838999609 0.6430978898007034
0.2 0.14 0.8 0.8 0.5888286440015631 0.6430978898007034
0.2 0.14 0.8 0.9 0.5193923407581086 0.6430978898007034
0.2 0.14 0.8 1.0 0.41466393122313405 0.6430978898007034
0.2 0.14 0.9 0.0 0.585336068776866 0.6430978898007034
0.2 0.14 0.9 0.1 0.6140093786635404 0.6430978898007034
0.2 0.14 0.9 0.2 0.6222401328644002 0.6430978898007034
0.2 0.14 0.9 0.3 0.6273202422821414 0.6430978898007034
0.2 0.14 0.9 0.4 0.6323026572880032 0.6430978898007034
0.2 0.14 0.9 0.5 0.6386527940601797 0.6430978898007034
0.2 0.14 0.9 0.6 0.641559202813599 0.6430978898007034
0.2 0.14 0.9

0.2 0.18 0.0 0.0 0.585336068776866 0.6430978898007034
0.2 0.18 0.0 0.1 0.6056809300508011 0.6430978898007034
0.2 0.18 0.0 0.2 0.6173554122704181 0.6430978898007034
0.2 0.18 0.0 0.3 0.6256105900742478 0.6430978898007034
0.2 0.18 0.0 0.4 0.6327178585384916 0.6430978898007034
0.2 0.18 0.0 0.5 0.6380910511918718 0.6430978898007034
0.2 0.18 0.0 0.6 0.629909144196952 0.6430978898007034
0.2 0.18 0.0 0.7 0.6143513091051191 0.6430978898007034
0.2 0.18 0.0 0.8 0.5790347792106292 0.6430978898007034
0.2 0.18 0.0 0.9 0.5117721766314967 0.6430978898007034
0.2 0.18 0.0 1.0 0.41466393122313405 0.6430978898007034
0.2 0.18 0.1 0.0 0.585336068776866 0.6430978898007034
0.2 0.18 0.1 0.1 0.6056809300508011 0.6430978898007034
0.2 0.18 0.1 0.2 0.6170379054318093 0.6430978898007034
0.2 0.18 0.1 0.3 0.6254152012504884 0.6430978898007034
0.2 0.18 0.1 0.4 0.6327667057444314 0.6430978898007034
0.2 0.18 0.1 0.5 0.6382131692067213 0.6430978898007034
0.2 0.18 0.1 0.6 0.6327911293474013 0.6430978898007034
0.2 0.18 0.1

0.2 0.2 0.3 0.0 0.585336068776866 0.6430978898007034
0.2 0.2 0.3 0.1 0.6042887846815163 0.6430978898007034
0.2 0.2 0.3 0.2 0.615767878077374 0.6430978898007034
0.2 0.2 0.3 0.3 0.6243894099257522 0.6430978898007034
0.2 0.2 0.3 0.4 0.6330842125830403 0.6430978898007034
0.2 0.2 0.3 0.5 0.6384574052364205 0.6430978898007034
0.2 0.2 0.3 0.6 0.6388237592809691 0.6430978898007034
0.2 0.2 0.3 0.7 0.6206037514654161 0.6430978898007034
0.2 0.2 0.3 0.8 0.5852627979679562 0.6430978898007034
0.2 0.2 0.3 0.9 0.5114790933958577 0.6430978898007034
0.2 0.2 0.3 1.0 0.41466393122313405 0.6430978898007034
0.2 0.2 0.4 0.0 0.585336068776866 0.6430978898007034
0.2 0.2 0.4 0.1 0.6049237983587339 0.6430978898007034
0.2 0.2 0.4 0.2 0.6167936694021102 0.6430978898007034
0.2 0.2 0.4 0.3 0.6243894099257522 0.6430978898007034
0.2 0.2 0.4 0.4 0.6338657678780774 0.6430978898007034
0.2 0.2 0.4 0.5 0.6389703008987886 0.6430978898007034
0.2 0.2 0.4 0.6 0.6405089878858929 0.6430978898007034
0.2 0.2 0.4 0.7 0.622923993747

In [ ]:
# 0.08 0.16 0.6 0.5 0.6430978898007034 0.6430978898007034
print(f"Best accuracy score: {best_res}\n\n" + 
      "Achieved with hyperparameters:\n" + 
      "\talpha1 = 0.08;\n" +   
      "\talpha2 = 0.16;\n" + 
      "\tcontribution weights of two models: 0.6;\n" +
      "\tthreshold for predict_proba(): 0.5;")

Best accuracy score: 0.6430978898007034

Achieved with hyperparameters:
	alpha1 = 0.08;
	alpha2 = 0.16;
	contribution weights of two models: 0.6;
	threshold for predict_proba(): 0.5;


Optional: check dependency of fine-graned threshold -> no significant improvement

In [ ]:
# best_res = None
# for alpha1 in [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]:
#     model1 = MultinomialNB(alpha=alpha1)  # TODO: check other alpha
#     model1.fit(train_features, y_train)
#     for alpha2 in [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]:
#         model2 = MultinomialNB(alpha=alpha2)  # TODO: check other alpha
#         model2.fit(train_features2, y_train)

#         for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#             q = 1 - p
#             probas1 = model1.predict_proba(test_features)[:, 1]  # take class 1
#             probas2 = model2.predict_proba(test_features2)[:, 1]  # take class 1

#             probas_sum = p * probas1 + q * probas2
#             for th in [0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49,
#                        0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60]:
#                 preds = (probas_sum >= th) * 1  # convert bool array to [0, 1]
#                 cur_res = accuracy_score(preds, y_test)
#                 if best_res is None or best_res < cur_res:
#                     best_res = cur_res
#                 print(alpha1, alpha2, p, th, cur_res, best_res)
                

In [ ]:
print("Best accuracy score: 0.6431467370066433\n\n" + 
      "Achieved with hyperparameters:\n" + 
      "\talpha1 = 0.08;\n" +   
      "\talpha2 = 0.12;\n" + 
      "\tcontribution weights of two models: 0.6;\n" +
      "\tthreshold for predict_proba(): 0.49;")

Best accuracy score: 0.6431467370066433

Achieved with hyperparameters:
	alpha1 = 0.08;
	alpha2 = 0.12;
	contribution weights of two models: 0.6;
	threshold for predict_proba(): 0.49;


## Check baseline approach (shuffled labels)

### Report for shuffled, all-0 and all-1 test labels

In [ ]:
model = MultinomialNB(alpha=0.1)
model.fit(train_features, y_train);

In [ ]:
y_pred = model.predict(test_features)

In [ ]:
rng = np.random.default_rng(seed=42)
y_pred2 = y_pred.copy()
rng.shuffle(y_pred2)
print(y_pred[:10], y_pred2[:10])
print(classification_report(y_test, y_pred2, digits=3))

[1 0 1 1 1 1 1 1 0 1] [1 1 1 1 0 0 1 1 1 1]
              precision    recall  f1-score   support

           0      0.422     0.207     0.277     16978
           1      0.587     0.799     0.677     23966

    accuracy                          0.554     40944
   macro avg      0.504     0.503     0.477     40944
weighted avg      0.519     0.554     0.511     40944



In [ ]:
y_pred3 = np.zeros_like(y_pred)
print(y_pred[:10], y_pred3[:10])
print(classification_report(y_test, y_pred3, digits=3))

[1 0 1 1 1 1 1 1 0 1] [0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0      0.415     1.000     0.586     16978
           1      0.000     0.000     0.000     23966

    accuracy                          0.415     40944
   macro avg      0.207     0.500     0.293     40944
weighted avg      0.172     0.415     0.243     40944



C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred4 = np.ones_like(y_pred)
print(y_pred[:10], y_pred4[:10])
print(classification_report(y_test, y_pred4, digits=3))

[1 0 1 1 1 1 1 1 0 1] [1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0      0.000     0.000     0.000     16978
           1      0.585     1.000     0.738     23966

    accuracy                          0.585     40944
   macro avg      0.293     0.500     0.369     40944
weighted avg      0.343     0.585     0.432     40944



C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Summary

In [ ]:
"""
Accuracy scores for the models on test data:  

    Baseline of Naive Bayes:  
        0.6200 

    Improved Naive Bayes:  
        0.6402
            - Restore stop words  
            - Append ticker name to the end of message  
            - Tuned hyperparameters alpha, ngram_range 
        0.6381
            - Trained on character ngrams only    
        0.6431
            - Blended (on words + on characters) with tuned hyperparameters:  

""";